## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,The Fairchild Condo\r3581 E Glencoe 501\rMiami,5/16/2023,58,"$6,600,000.00",3974,"$2,058.64",Robin Depeder,Douglas Elliman,Marco Tine,Casa Collection Realty,The Fairchild Condo 3581 E Glencoe 501 Miami,The Fairchild Condo,3581 E Glencoe 501 Miami,"(25.7401591, -80.21738979999999)",25.740159,-80.217390
2,St Regis\r9703 Collins Ave 2200\rBal Harbour,5/17/2023,193,"$6,300,000.00",2721,"$2,315.33",Alan Chocron,"Compass Florida, LLC",Gregory Gringruz,The Island,St Regis 9703 Collins Ave 2200 Bal Harbour,St Regis,9703 Collins Ave 2200 Bal Harbour,"(25.8888332, -80.1225323)",25.888833,-80.122532
3,Murano at Portofino Condo\r1000 S Pointe Dr 19...,5/18/2023,76,"$5,200,000.00",2618,"$1,986.25",Daniel Hechtkopf,Douglas Elliman,Mark Zilbert,Zilbert,Murano at Portofino Condo 1000 S Pointe Dr 190...,Murano at Portofino Condo,1000 S Pointe Dr 1902 Miami Beach,"(25.7685326, -80.13755619999999)",25.768533,-80.137556
4,57 Ocean Condo\r5775 Collins Ave 1406\rMiami B...,5/15/2023,56,"$3,550,000.00",1276,"$2,782.13",Christopher Jude,Fortune Christie's Int'l Real Estate,David Nguah,Douglas Elliman,57 Ocean Condo 5775 Collins Ave 1406 Miami Beach,,57 Ocean Condo 5775 Collins Ave 1406 Miami Beach,"(25.8392568, -80.12039469999999)",25.839257,-80.120395
5,The Ocean Club\r785 Crandon Blvd 1702\rKey Bis...,5/15/2023,28,"$2,700,000.00",2100,"$1,285.71",Douglas Kinsley,Fortune Christie's Int'l Real Estate,Douglas Kinsley,Fortune Christie's Int'l Real Estate,The Ocean Club 785 Crandon Blvd 1702 Key Biscayne,The Ocean Club,785 Crandon Blvd 1702 Key Biscayne,"(25.6875627, -80.1581203)",25.687563,-80.158120
6,Trump Royale\r18201 Collins Ave 3504\rSunny Is...,5/16/2023,203,"$2,130,000.00",2174,$979.76,Fernando Alpern,EXP Realty LLC,Olga Zaurova,"Big International Realty, Inc.",Trump Royale 18201 Collins Ave 3504 Sunny Isle...,Trump Royale,18201 Collins Ave 3504 Sunny Isles Beach,"(25.9449899, -80.12008929999999)",25.944990,-80.120089
7,The Peninsula Condo\r3201 NE 183 St 2307\rAven...,5/19/2023,196,"$1,828,000.00",3274,$558.34,Richard Goihman,Douglas Elliman,Anastasia Gray,EXP Realty LLC,The Peninsula Condo 3201 NE 183 St 2307 Aventura,The Peninsula Condo,3201 NE 183 St 2307 Aventura,"(25.9456839, -80.13577169999999)",25.945684,-80.135772
8,Grove Towers Condo\r2843 S Bayshore Dr P3E\rMiami,5/15/2023,193,"$1,700,000.00",1200,"$1,416.67",Isabelle Andrews PA,BHHS EWM Realty,Laura Anderson,"Compass Florida, LLC",Grove Towers Condo 2843 S Bayshore Dr P3E Miami,Grove Towers Condo,2843 S Bayshore Dr P3E Miami,"(25.7281817, -80.23897199999999)",25.728182,-80.238972
9,Armitage Condo\r730 3rd St 203\rMiami Beach,5/17/2023,66,"$1,600,000.00",1750,$914.29,Dayanne Costa,"Compass Florida, LLC",Nathan Shore,Douglas Elliman,Armitage Condo 730 3rd St 203 Miami Beach,Armitage Condo,730 3rd St 203 Miami Beach,"(25.7721593, -80.1357599)",25.772159,-80.135760
10,0Hamptons South\r20201 E Country Club Dr 2702\...,5/16/2023,122,"$1,587,500.00",3262,$486.66,Stuart Berger,Coldwell Banker Realty,Alberto Esquenazi,Marina Realty Group,0Hamptons South 20201 E Country Club Dr 2702 A...,,0Hamptons South 20201 E Country Club Dr 2702 A...,"(25.9649419, -80.1252236)",25.964942,-80.125224


In [10]:
df.at[4,'building_name']=('57 Ocean')
df.at[4,'address_only']=('5775 Collins Ave 1406 Miami Beach')

df.at[10,'building_name']=('Hamptons South')
df.at[10,'address_only']=('20201 E Country Club Dr')

## Format Data

In [11]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [12]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [13]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [14]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [15]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [16]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('May 14th - May 20th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [17]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon,int_Sale_Price,RANK,COLOR
1,The Fairchild Condo\r3581 E Glencoe 501\rMiami,5/16/2023,58,"$6,600,000.00",3974,2058.64,Robin Depeder,Douglas Elliman,Marco Tine,Casa Collection Realty,The Fairchild Condo 3581 E Glencoe 501 Miami,The Fairchild Condo,3581 E Glencoe 501 Miami,"(25.7401591, -80.21738979999999)",25.740159,-80.217390,6600000.0,1,orange
2,St Regis\r9703 Collins Ave 2200\rBal Harbour,5/17/2023,193,"$6,300,000.00",2721,2315.33,Alan Chocron,"Compass Florida, LLC",Gregory Gringruz,The Island,St Regis 9703 Collins Ave 2200 Bal Harbour,St Regis,9703 Collins Ave 2200 Bal Harbour,"(25.8888332, -80.1225323)",25.888833,-80.122532,6300000.0,2,blue
3,Murano at Portofino Condo\r1000 S Pointe Dr 19...,5/18/2023,76,"$5,200,000.00",2618,1986.25,Daniel Hechtkopf,Douglas Elliman,Mark Zilbert,Zilbert,Murano at Portofino Condo 1000 S Pointe Dr 190...,Murano at Portofino Condo,1000 S Pointe Dr 1902 Miami Beach,"(25.7685326, -80.13755619999999)",25.768533,-80.137556,5200000.0,3,blue
4,57 Ocean Condo\r5775 Collins Ave 1406\rMiami B...,5/15/2023,56,"$3,550,000.00",1276,2782.13,Christopher Jude,Fortune Christie's Int'l Real Estate,David Nguah,Douglas Elliman,57 Ocean Condo 5775 Collins Ave 1406 Miami Beach,57 Ocean,5775 Collins Ave 1406 Miami Beach,"(25.8392568, -80.12039469999999)",25.839257,-80.120395,3550000.0,4,blue
5,The Ocean Club\r785 Crandon Blvd 1702\rKey Bis...,5/15/2023,28,"$2,700,000.00",2100,1285.71,Douglas Kinsley,Fortune Christie's Int'l Real Estate,Douglas Kinsley,Fortune Christie's Int'l Real Estate,The Ocean Club 785 Crandon Blvd 1702 Key Biscayne,The Ocean Club,785 Crandon Blvd 1702 Key Biscayne,"(25.6875627, -80.1581203)",25.687563,-80.158120,2700000.0,5,blue
6,Trump Royale\r18201 Collins Ave 3504\rSunny Is...,5/16/2023,203,"$2,130,000.00",2174,979.76,Fernando Alpern,EXP Realty LLC,Olga Zaurova,"Big International Realty, Inc.",Trump Royale 18201 Collins Ave 3504 Sunny Isle...,Trump Royale,18201 Collins Ave 3504 Sunny Isles Beach,"(25.9449899, -80.12008929999999)",25.944990,-80.120089,2130000.0,6,blue
7,The Peninsula Condo\r3201 NE 183 St 2307\rAven...,5/19/2023,196,"$1,828,000.00",3274,558.34,Richard Goihman,Douglas Elliman,Anastasia Gray,EXP Realty LLC,The Peninsula Condo 3201 NE 183 St 2307 Aventura,The Peninsula Condo,3201 NE 183 St 2307 Aventura,"(25.9456839, -80.13577169999999)",25.945684,-80.135772,1828000.0,7,blue
8,Grove Towers Condo\r2843 S Bayshore Dr P3E\rMiami,5/15/2023,193,"$1,700,000.00",1200,1416.67,Isabelle Andrews PA,BHHS EWM Realty,Laura Anderson,"Compass Florida, LLC",Grove Towers Condo 2843 S Bayshore Dr P3E Miami,Grove Towers Condo,2843 S Bayshore Dr P3E Miami,"(25.7281817, -80.23897199999999)",25.728182,-80.238972,1700000.0,8,blue
9,Armitage Condo\r730 3rd St 203\rMiami Beach,5/17/2023,66,"$1,600,000.00",1750,914.29,Dayanne Costa,"Compass Florida, LLC",Nathan Shore,Douglas Elliman,Armitage Condo 730 3rd St 203 Miami Beach,Armitage Condo,730 3rd St 203 Miami Beach,"(25.7721593, -80.1357599)",25.772159,-80.135760,1600000.0,9,blue
10,0Hamptons South\r20201 E Country Club Dr 2702\...,5/16/2023,122,"$1,587,500.00",3262,486.66,Stuart Berger,Coldwell Banker Realty,Alberto Esquenazi,Marina Realty Group,0Hamptons South 20201 E Country Club Dr 2702 A...,Hamptons South,20201 E Country Club Dr,"(25.9649419, -80.1252236)",25.964942,-80.125224,1587500.0,10,blue


In [18]:
m.save('index.html')

## Data snagger

In [19]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [20]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
The Fairchild Condo , 3581 E Glencoe 501 Miami | Price $6,600,000 | $2,059 psf | Listing agent: Robin Depeder with Douglas Elliman | Buyer's agent: Marco Tine with Casa Collection Realty | Days on market: 58
Least Expensive
Hamptons South, 20201 E Country Club Dr | Price $1,587,500 | $487 psf | Listing agent: Stuart Berger with Coldwell Banker Realty | Buyer's agent: Alberto Esquenazi with Marina Realty Group | Days on market: 122
Highest Price Per Square Foot
57 Ocean, 5775 Collins Ave 1406 Miami Beach | Price $3,550,000 | $2,782 psf | Listing agent: Christopher Jude with Fortune Christie's Int'l Real Estate | Buyer's agent: David Nguah with Douglas Elliman | Days on market: 56
Lowest Price Per Square Foot
Hamptons South, 20201 E Country Club Dr | Price $1,587,500 | $487 psf | Listing agent: Stuart Berger with Coldwell Banker Realty | Buyer's agent: Alberto Esquenazi with Marina Realty Group | Days on market: 122
Most Days on Market
Trump Royale , 18201 Collins Ave 3504

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [21]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [22]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_05202023
